In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
from librairies.bt import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter

scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait()

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

In [2]:
####### CHOIX DE LA TIME FRAME
_period = 'm15'

In [3]:
%%time
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = list(sorted(set(get_ticker_list())))
TICKER_LIST = ['EUR/USD']


Récupération des tickers
CPU times: user 746 µs, sys: 940 µs, total: 1.69 ms
Wall time: 921 µs


In [4]:
#############################
##### F E A T U R I N G #####
#############################

In [5]:
%%time
%%time
df_all = joblib.load('JOBLIB/Built_bases/df_all')
daily_all = joblib.load('JOBLIB/Built_bases/daily_all')
weekly_all = joblib.load('JOBLIB/Built_bases/weekly_all')
df_all.shape,daily_all.shape,weekly_all.shape

CPU times: user 3.33 s, sys: 4.56 s, total: 7.89 s
Wall time: 7.97 s
CPU times: user 3.33 s, sys: 4.56 s, total: 7.89 s
Wall time: 7.97 s


((11069853, 56), (119160, 7), (23981, 19))

In [6]:
%%time
#### S T R A T E G Y

##### CONDITIONS LONG
_condition_1 = (df_all.slow_K5 < 20) & (df_all.slow_K5.shift(1) < df_all.slow_D5.shift(1)) & (df_all.slow_K5 > df_all.slow_D5)

##### CONDITIONS SHORT
_condition_1_bar = (df_all.slow_K5 > 80) & (df_all.slow_K5.shift(1) > df_all.slow_D5.shift(1)) & (df_all.slow_K5 < df_all.slow_D5)

##### 1 condition
df_all['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))

CPU times: user 613 ms, sys: 74.7 ms, total: 688 ms
Wall time: 401 ms


In [7]:
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = '2010-01-01'
_year_top = '2017-12-31'
_nb_bougie_exit = 9600000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 1
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target = 0.004
_exposure = 10
_size = 50000
_sl = 0.002
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
##### Backtest Over Night
TRACKER = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)


EUR/USD
Librairies imported

Début des opérations horodatée à 2021-02-04 19:27:34.099499

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 192148/192148 [00:08<00:00, 23198.28it/s]
 Pour le ticker  EUR/USD 
 
Les gains faramineux s'élèvent à : $  -46498.0 !. En  2926   transactions.
 
Nbre de winners : 1253 
 
Nbre de loosers : 1673 
Temps d'excution du BT 0:00:14.788620
 EUR/USD  results 
 Tested Period 2010-01-01  à 2017-12-31 
 Total Number of trades 2926 
Started Cash : 50000
P&L in currency:  -46498.0$ 
P&L in %:  -23.25% 
Average trade duration 29.7
# Winners  1253.0
# Loosers  1673.0
Cumulated gains 210796.5
Cumulated losses -257294.5
 PROFIT FACTOR :  0.82 
 Winners Ratio : 42.82 % 
Average Winners 168.37
% Average Winners 0.08
Average Loosers -153.79
% Average Loosers -0.08
Average pnl -15.89
% Average pnl -0.02
Number of opened trades 2926
Number of closed trades 2926
Max Expos

In [11]:
%%time

features = featuring(df_all)
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0)
features['Signal'] = np.where(((features.Valid==1)&(features.Signal==1)),1,np.where(((features.Valid==1)&(features.Signal==-1)),-1,0))
df_all['Signal'] = np.where(df_all.index.isin(features[features.Signal==1].index),1,np.where(df_all.index.isin(features[features.Signal==-1].index),-1,0))
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = '2010-01-01'
_year_top = '2017-12-31'
_nb_bougie_exit = 9600000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 1
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target = 0.004
_exposure = 10
_size = 50000
_sl = 0.002
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only

##### Backtest Over Night
TRACKER = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

EUR/USD
Librairies imported

Début des opérations horodatée à 2021-02-04 19:29:45.222224

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 192148/192148 [00:04<00:00, 41947.94it/s]
 Pour le ticker  EUR/USD 
 
Les gains faramineux s'élèvent à : $  158679.5 !. En  552   transactions.
 
Nbre de winners : 552 
 
Nbre de loosers : 0 
Temps d'excution du BT 0:00:10.989167
 EUR/USD  results 
 Tested Period 2010-01-01  à 2017-12-31 
 Total Number of trades 552 
Started Cash : 50000
P&L  in currency:  158679.5$ 
P&L in %:  79.34% 
Average trade duration 57.92
# Winners  552.0
# Loosers  0.0
Cumulated gains 158679.5
Cumulated losses 0.0
 PROFIT FACTOR :  inf 
 Winners Ratio : 100.0 % 
Average Winners 287.46
% Average Winners 0.14
Pas de loosers
Average pnl 287.46
% Average pnl 0.22
Number of opened trades 552
Number of closed trades 552
Max Exposure 1 x  50000 =  50000 $
CPU times: user 14 s, sys:

In [12]:
%%time
df_all = joblib.load('JOBLIB/Built_bases/df_all')
df_all['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
features = featuring(df_all)
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

CPU times: user 8.21 s, sys: 15.7 s, total: 23.9 s
Wall time: 20.1 s


In [13]:
%%time
# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

CPU times: user 104 ms, sys: 9.35 ms, total: 113 ms
Wall time: 55.4 ms


In [14]:
%%time
# Make our choice for the split, compliant to our backtest
_start = '2010-01-01' # start the train there
_mid = '2015-08-31' # stop the train and begin the test there
_stop = '2017-12-31' # stop the test there. After that, it is kept for oos

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [15]:
%%time
# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]
# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

CPU times: user 3.34 s, sys: 2.47 s, total: 5.81 s
Wall time: 5.8 s


In [16]:
%%time
# Let's create our train & test dataframes
features_train = features[((features.index>=_start)&(features.index<_mid))] # we exclude _mid
features_test = features[((features.index>=_mid)&(features.index<_stop))] # we exclude _stop

CPU times: user 21.7 ms, sys: 6.84 ms, total: 28.6 ms
Wall time: 27.3 ms


In [17]:
%%time
# import the classification tool
from pycaret.classification import *

CPU times: user 453 ms, sys: 108 ms, total: 561 ms
Wall time: 750 ms


In [18]:
%%time
# Let's try a naive classification with no scaling as if it was balanced
naive = setup(data = features_train.drop(['TRACKER'],axis=1), test_data = features_test.drop(['TRACKER'],axis=1), target = 'Valid', session_id=26) # ,fix_imbalance=True,fix_imbalance_method=True)
best_model = compare_models(sort='F1',fold=5)
engine.say("Naive Classification with date and symbol done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9971,0.9954,0.2190,0.5562,0.3030,0.3019,0.3393,19.2300
dt,Decision Tree Classifier,0.9952,0.6572,0.3171,0.2668,0.2841,0.2818,0.2856,0.5720
xgboost,Extreme Gradient Boosting,0.9972,0.9945,0.1886,0.5740,0.2747,0.2737,0.3210,6.1520
lda,Linear Discriminant Analysis,0.9878,0.9906,0.7383,0.1614,0.2642,0.2606,0.3409,0.6300
lightgbm,Light Gradient Boosting Machine,0.9941,0.9080,0.3275,0.2283,0.2586,0.2560,0.2652,0.5600
ada,Ada Boost Classifier,0.9969,0.9921,0.1183,0.4916,0.1791,0.1781,0.2295,3.8900
qda,Quadratic Discriminant Analysis,0.9853,0.8919,0.6569,0.0998,0.1726,0.1694,0.2518,0.6540
rf,Random Forest Classifier,0.9972,0.9923,0.0930,0.6808,0.1509,0.1503,0.2315,1.7620
gbc,Gradient Boosting Classifier,0.9967,0.8240,0.1056,0.2953,0.1460,0.1449,0.1685,19.7800
et,Extra Trees Classifier,0.9970,0.9909,0.0502,0.5274,0.0829,0.0823,0.1383,0.7860


CPU times: user 5min 9s, sys: 26.3 s, total: 5min 36s
Wall time: 5min 36s


In [21]:
best_model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'Logloss',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 26,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'Logloss',
 'learning_rate': 0.08382000029087067,
 'score_function': 'Cosine',
 'task_type': 'CPU',
 'leaf_estimation_iterations': 10,
 'bo

In [22]:
%%time
# Let's try a naive classification with no scaling as if it was balanced
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26) # ,fix_imbalance=True,fix_imbalance_method=True)
best_model = compare_models(sort='F1',fold=5)
engine.say("Naive Classification without date and symbol done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9971,0.9952,0.2165,0.5488,0.3030,0.3019,0.3381,19.1160
dt,Decision Tree Classifier,0.9953,0.6560,0.3148,0.2693,0.2862,0.2840,0.2868,0.4580
xgboost,Extreme Gradient Boosting,0.9971,0.9944,0.1938,0.5495,0.2789,0.2778,0.3195,5.0860
lda,Linear Discriminant Analysis,0.9878,0.9906,0.7383,0.1614,0.2642,0.2606,0.3409,0.4960
lightgbm,Light Gradient Boosting Machine,0.9946,0.8848,0.2772,0.2198,0.2371,0.2345,0.2401,0.7880
rf,Random Forest Classifier,0.9973,0.9906,0.1384,0.7038,0.2236,0.2228,0.3024,1.9980
qda,Quadratic Discriminant Analysis,0.9792,0.9879,0.9273,0.1206,0.2122,0.2081,0.3283,0.2860
ada,Ada Boost Classifier,0.9970,0.9924,0.1308,0.5246,0.1955,0.1946,0.2476,3.6460
gbc,Gradient Boosting Classifier,0.9967,0.8241,0.1106,0.3138,0.1519,0.1508,0.1766,18.2920
et,Extra Trees Classifier,0.9971,0.9908,0.0804,0.6914,0.1310,0.1304,0.2093,0.6540


CPU times: user 5min 4s, sys: 24.6 s, total: 5min 29s
Wall time: 5min


In [24]:
best_model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'Logloss',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 26,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'Logloss',
 'learning_rate': 0.08382000029087067,
 'score_function': 'Cosine',
 'task_type': 'CPU',
 'leaf_estimation_iterations': 10,
 'bo

In [25]:
%%time
# Let's try a naive classification with no scaling as if it was balanced
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol','Signal'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26) # ,fix_imbalance=True,fix_imbalance_method=True)
best_model = compare_models(sort='F1',fold=5)
engine.say("Naive Classification without signal, date and symbol done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9940,0.7365,0.0527,0.0559,0.0433,0.0407,0.0455,0.2760
lightgbm,Light Gradient Boosting Machine,0.9937,0.7554,0.0427,0.0399,0.0405,0.0376,0.0379,0.6160
gbc,Gradient Boosting Classifier,0.9950,0.8078,0.0176,0.0299,0.0207,0.0185,0.0198,17.7500
dt,Decision Tree Classifier,0.9851,0.5115,0.0352,0.0104,0.0158,0.0112,0.0131,1.3680
xgboost,Extreme Gradient Boosting,0.9970,0.8125,0.0050,0.2400,0.0097,0.0096,0.0321,6.3220
catboost,CatBoost Classifier,0.9969,0.8242,0.0050,0.0778,0.0089,0.0086,0.0177,18.2980
lr,Logistic Regression,0.4757,0.5691,0.6649,0.0037,0.0074,0.0016,0.0152,0.1500
nb,Naive Bayes,0.0079,0.4922,0.9975,0.0029,0.0059,0.0000,0.0021,0.0800
qda,Quadratic Discriminant Analysis,0.0136,0.5002,0.9899,0.0029,0.0058,0.0000,-0.0006,0.5740
svm,SVM - Linear Kernel,0.5034,0.0000,0.4506,0.0026,0.0052,-0.0006,-0.0049,0.2160


CPU times: user 9.78 s, sys: 1.19 s, total: 11 s
Wall time: 4min 55s


In [27]:
best_model # .get_all_params()

AttributeError: 'LinearDiscriminantAnalysis' object has no attribute 'get_all_params'

In [17]:
%%time
# Let's try to tune some hyperparameters. First, let's see if fix imbalance improves.
imbalanced = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26,fix_imbalance=True,fix_imbalance_method=None)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Naive Classification imbalanced done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9868,0.9910,0.6556,0.2941,0.4048,0.3991,0.4329,0.8060
xgboost,Extreme Gradient Boosting,0.9868,0.9889,0.6189,0.2865,0.3898,0.3841,0.4144,4.3640
catboost,CatBoost Classifier,0.9857,0.9910,0.6776,0.2776,0.3933,0.3874,0.4277,12.7400
dt,Decision Tree Classifier,0.9884,0.7104,0.4286,0.2751,0.3336,0.3281,0.3370,1.1360
rf,Random Forest Classifier,0.9870,0.9903,0.5349,0.2677,0.3553,0.3494,0.3717,3.3420
et,Extra Trees Classifier,0.9849,0.9905,0.6628,0.2617,0.3742,0.3680,0.4099,1.0140
gbc,Gradient Boosting Classifier,0.9800,0.9911,0.8605,0.2348,0.3687,0.3619,0.4433,10.2500
ridge,Ridge Classifier,0.9773,0.0000,1.0000,0.2312,0.3752,0.3682,0.4749,0.0640
lda,Linear Discriminant Analysis,0.9773,0.9885,1.0000,0.2312,0.3752,0.3682,0.4749,0.1960
ada,Ada Boost Classifier,0.9780,0.9885,0.9486,0.2308,0.3709,0.3639,0.4618,2.2280


CPU times: user 16.6 s, sys: 1.12 s, total: 17.7 s
Wall time: 3min 12s


In [18]:
%%time
# Let's play with scaling . First, normalization zscore
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, normalize=True, normalize_method='zscore')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with zscore done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9924,0.9914,0.1393,0.3754,0.1968,0.1938,0.2212,1.7420
catboost,CatBoost Classifier,0.9923,0.9924,0.1686,0.3740,0.2279,0.2246,0.2450,13.2240
rf,Random Forest Classifier,0.9926,0.9861,0.0440,0.3629,0.0724,0.0707,0.1160,0.4860
et,Extra Trees Classifier,0.9924,0.9846,0.0366,0.3492,0.0629,0.0611,0.1061,0.1460
gbc,Gradient Boosting Classifier,0.9921,0.9914,0.0768,0.3296,0.1190,0.1164,0.1515,4.9680
svm,SVM - Linear Kernel,0.9931,0.0000,0.0073,0.3000,0.0142,0.0137,0.0451,0.0640
dt,Decision Tree Classifier,0.9901,0.6495,0.3042,0.2935,0.2964,0.2915,0.2927,0.0620
ada,Ada Boost Classifier,0.9920,0.9378,0.1098,0.2864,0.1537,0.1505,0.1701,1.0020
lightgbm,Light Gradient Boosting Machine,0.9907,0.9906,0.1940,0.2681,0.2233,0.2188,0.2226,0.6340
lda,Linear Discriminant Analysis,0.9773,0.9878,1.0000,0.2312,0.3752,0.3682,0.4749,0.1000


CPU times: user 39 s, sys: 1.51 s, total: 40.5 s
Wall time: 2min 33s


In [19]:
%%time
# Let's play with scaling . First, normalization minmax
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, normalize=True, normalize_method='minmax')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with minmax done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9923,0.9924,0.1830,0.3928,0.2444,0.2410,0.2614,13.1400
rf,Random Forest Classifier,0.9925,0.9863,0.0366,0.3583,0.0627,0.0609,0.1056,0.5220
gbc,Gradient Boosting Classifier,0.9921,0.9914,0.0877,0.3316,0.1332,0.1304,0.1629,4.9240
xgboost,Extreme Gradient Boosting,0.9921,0.9912,0.1430,0.3288,0.1966,0.1933,0.2116,1.7260
lightgbm,Light Gradient Boosting Machine,0.9908,0.9879,0.2236,0.2894,0.2492,0.2447,0.2483,0.8240
knn,K Neighbors Classifier,0.9918,0.8659,0.1316,0.2861,0.1789,0.1754,0.1895,3.3500
dt,Decision Tree Classifier,0.9899,0.6403,0.2859,0.2753,0.2778,0.2728,0.2741,0.0620
ada,Ada Boost Classifier,0.9919,0.9413,0.0989,0.2656,0.1415,0.1383,0.1568,1.1700
et,Extra Trees Classifier,0.9922,0.9857,0.0440,0.2524,0.0723,0.0701,0.1002,0.1660
nb,Naive Bayes,0.9773,0.9885,1.0000,0.2312,0.3752,0.3682,0.4749,0.0460


CPU times: user 3min 8s, sys: 8.89 s, total: 3min 17s
Wall time: 2min 33s


In [20]:
%%time
# Let's play with scaling . First, normalization maxabs
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, normalize=True, normalize_method='maxabs')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with maxabs done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9929,0.9913,0.1722,0.4628,0.2446,0.2417,0.2748,1.7960
catboost,CatBoost Classifier,0.9923,0.9921,0.1613,0.3878,0.2209,0.2177,0.2424,13.3980
lightgbm,Light Gradient Boosting Machine,0.9917,0.9904,0.2123,0.3279,0.2548,0.2509,0.2583,0.9960
rf,Random Forest Classifier,0.9925,0.9877,0.0367,0.3276,0.0623,0.0606,0.1013,0.5220
dt,Decision Tree Classifier,0.9898,0.6657,0.3372,0.3006,0.3149,0.3099,0.3118,0.0700
gbc,Gradient Boosting Classifier,0.9921,0.9911,0.0770,0.2902,0.1123,0.1097,0.1387,4.9880
knn,K Neighbors Classifier,0.9916,0.8622,0.1355,0.2773,0.1804,0.1767,0.1890,2.4780
et,Extra Trees Classifier,0.9923,0.9857,0.0477,0.2550,0.0748,0.0727,0.1022,0.1480
nb,Naive Bayes,0.9773,0.9885,1.0000,0.2312,0.3752,0.3682,0.4749,0.0360
lda,Linear Discriminant Analysis,0.9773,0.9878,1.0000,0.2312,0.3752,0.3682,0.4749,0.1060


CPU times: user 40.1 s, sys: 1.56 s, total: 41.6 s
Wall time: 2min 20s


In [21]:
%%time
# Let's play with scaling . Normalization robust
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, normalize=True, normalize_method='robust')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with robust done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9924,0.9923,0.1576,0.4211,0.2220,0.2189,0.2494,13.5320
rf,Random Forest Classifier,0.9927,0.9843,0.0367,0.3486,0.0627,0.0612,0.1038,0.5240
xgboost,Extreme Gradient Boosting,0.9920,0.9911,0.1466,0.3353,0.1982,0.1948,0.2145,1.7640
et,Extra Trees Classifier,0.9925,0.9849,0.0402,0.3152,0.0688,0.0669,0.1070,0.1660
lightgbm,Light Gradient Boosting Machine,0.9910,0.9905,0.2201,0.2950,0.2489,0.2445,0.2488,0.8480
gbc,Gradient Boosting Classifier,0.9921,0.9913,0.0659,0.2906,0.1010,0.0984,0.1298,5.2880
dt,Decision Tree Classifier,0.9901,0.6460,0.2972,0.2859,0.2890,0.2840,0.2853,0.0640
ada,Ada Boost Classifier,0.9919,0.9368,0.1098,0.2773,0.1530,0.1498,0.1679,1.1300
lda,Linear Discriminant Analysis,0.9773,0.9878,1.0000,0.2312,0.3752,0.3682,0.4749,0.1020
qda,Quadratic Discriminant Analysis,0.9845,0.7916,0.5927,0.1445,0.2322,0.2281,0.2892,0.0760


CPU times: user 3min 11s, sys: 9.7 s, total: 3min 21s
Wall time: 2min 39s


In [15]:
%%time
# Let's play with scaling . Transformation yeo-johnson
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, transformation=True, transformation_method='yeo-johnson')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with yeo-johnson done")
engine.runAndWait()

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:44:40
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
FEMA_21,Numeric
FEMA_8,Numeric
FADRLo,Numeric
FADRHi,Numeric
FRVI40,Numeric
FRVI60,Numeric
FONLOSMA5,Numeric
FONHISMA5,Numeric
FONLOSMA21,Numeric
FONHISMA21,Numeric


ValueError: Input contains infinity or a value too large for dtype('float32').

In [16]:
%%time
# Let's play with scaling . Transformation quantile
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, transformation=True, transformation_method='quantile')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with quantile done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9931,0.9892,0.0295,0.4600,0.0467,0.0460,0.0894,0.7180
xgboost,Extreme Gradient Boosting,0.9926,0.9915,0.1540,0.4048,0.2210,0.2180,0.2451,1.7420
et,Extra Trees Classifier,0.9927,0.9907,0.0367,0.3917,0.0629,0.0613,0.1102,0.1760
catboost,CatBoost Classifier,0.9923,0.9923,0.1467,0.3739,0.2061,0.2029,0.2279,13.6100
lightgbm,Light Gradient Boosting Machine,0.9913,0.9908,0.2237,0.3151,0.2589,0.2547,0.2598,0.2780
gbc,Gradient Boosting Classifier,0.9920,0.9912,0.0878,0.3076,0.1295,0.1267,0.1559,5.4720
dt,Decision Tree Classifier,0.9900,0.6586,0.3226,0.2937,0.3052,0.3002,0.3017,0.3200
rf,Random Forest Classifier,0.9927,0.9831,0.0330,0.2738,0.0560,0.0545,0.0885,0.5180
nb,Naive Bayes,0.9773,0.9885,1.0000,0.2312,0.3752,0.3682,0.4749,0.3180
lda,Linear Discriminant Analysis,0.9773,0.9884,1.0000,0.2312,0.3752,0.3682,0.4749,0.1020


CPU times: user 10 s, sys: 1.54 s, total: 11.5 s
Wall time: 2min 35s


In [40]:
%%time
# Let's see if combining normalization maxabs and transfromation quantile is better
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1), target = 'Valid', session_id=26, normalize=True, normalize_method='maxabs', transformation=True, transformation_method='quantile')
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with maxabs and quantile done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9931,0.9892,0.0295,0.4600,0.0467,0.0460,0.0894,0.2300
catboost,CatBoost Classifier,0.9924,0.9922,0.1615,0.4180,0.2225,0.2195,0.2499,13.5760
rf,Random Forest Classifier,0.9928,0.9849,0.0441,0.4048,0.0745,0.0731,0.1221,0.5340
xgboost,Extreme Gradient Boosting,0.9925,0.9912,0.1466,0.3777,0.2018,0.1989,0.2256,2.0500
lightgbm,Light Gradient Boosting Machine,0.9911,0.9871,0.2016,0.2953,0.2368,0.2325,0.2382,1.1200
et,Extra Trees Classifier,0.9927,0.9902,0.0366,0.2911,0.0625,0.0610,0.0968,0.2060
dt,Decision Tree Classifier,0.9900,0.6496,0.3044,0.2908,0.2933,0.2884,0.2905,0.0640
gbc,Gradient Boosting Classifier,0.9919,0.9913,0.0842,0.2884,0.1221,0.1193,0.1462,5.7840
ada,Ada Boost Classifier,0.9916,0.9332,0.0954,0.2388,0.1323,0.1289,0.1445,1.2420
nb,Naive Bayes,0.9773,0.9885,1.0000,0.2312,0.3752,0.3682,0.4749,0.0520


CPU times: user 11.5 s, sys: 1.2 s, total: 12.7 s
Wall time: 2min 41s


In [41]:
%%time
# Let's play with interactions
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, feature_interaction=True, feature_ratio=False,interaction_threshold=0.01)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9925,0.9924,0.1650,0.4017,0.2312,0.2281,0.2525,14.3780
xgboost,Extreme Gradient Boosting,0.9925,0.9920,0.1797,0.3907,0.2434,0.2401,0.2599,2.3360
lightgbm,Light Gradient Boosting Machine,0.9915,0.9909,0.2415,0.3723,0.2869,0.2828,0.2925,1.1220
ada,Ada Boost Classifier,0.9919,0.9425,0.2121,0.3677,0.2668,0.2630,0.2742,1.2600
rf,Random Forest Classifier,0.9928,0.9920,0.0551,0.3497,0.0906,0.0889,0.1293,0.3800
gbc,Gradient Boosting Classifier,0.9915,0.9919,0.2090,0.3240,0.2506,0.2465,0.2543,5.8920
lda,Linear Discriminant Analysis,0.9878,0.8957,0.6414,0.3106,0.4165,0.4112,0.4400,0.1860
et,Extra Trees Classifier,0.9927,0.9913,0.0403,0.2859,0.0676,0.0660,0.1016,0.2060
dt,Decision Tree Classifier,0.9896,0.6618,0.3296,0.2840,0.3022,0.2970,0.2993,0.1320
knn,K Neighbors Classifier,0.9922,0.8255,0.0806,0.2526,0.1215,0.1187,0.1390,0.2860


CPU times: user 4min 26s, sys: 11.5 s, total: 4min 38s
Wall time: 2min 47s


In [17]:
%%time
# Let's play with interactions
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, feature_interaction=True, feature_ratio=True,interaction_threshold=0.01)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9931,0.9922,0.0880,0.4746,0.1433,0.1415,0.1965,0.4460
catboost,CatBoost Classifier,0.9927,0.9924,0.1430,0.4129,0.2043,0.2016,0.2341,15.1560
lightgbm,Light Gradient Boosting Machine,0.9916,0.9910,0.2419,0.3551,0.2832,0.2791,0.2866,0.7460
gbc,Gradient Boosting Classifier,0.9921,0.9807,0.1576,0.3547,0.2129,0.2095,0.2298,7.8160
xgboost,Extreme Gradient Boosting,0.9922,0.9920,0.1723,0.3481,0.2266,0.2232,0.2391,2.7620
ada,Ada Boost Classifier,0.9916,0.9781,0.2127,0.3324,0.2553,0.2514,0.2598,1.5700
knn,K Neighbors Classifier,0.9923,0.8245,0.1065,0.3305,0.1549,0.1521,0.1795,0.6660
et,Extra Trees Classifier,0.9925,0.9913,0.0514,0.3233,0.0847,0.0827,0.1220,0.2400
lda,Linear Discriminant Analysis,0.9882,0.9164,0.6306,0.3155,0.4182,0.4129,0.4395,0.3120
dt,Decision Tree Classifier,0.9899,0.6564,0.3184,0.2853,0.2997,0.2947,0.2957,0.2460


CPU times: user 2min 24s, sys: 5.74 s, total: 2min 30s
Wall time: 3min


In [18]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, feature_interaction=True, feature_ratio=True,interaction_threshold=0.1)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9928,0.9916,0.1725,0.4376,0.2438,0.2409,0.2693,110.3180
rf,Random Forest Classifier,0.9926,0.9897,0.0771,0.4175,0.1187,0.1167,0.1651,3.7240
catboost,CatBoost Classifier,0.9927,0.9921,0.0843,0.4104,0.1352,0.1330,0.1774,116.9280
et,Extra Trees Classifier,0.9927,0.9874,0.0589,0.3924,0.0951,0.0933,0.1410,1.5960
lightgbm,Light Gradient Boosting Machine,0.9917,0.9895,0.1943,0.3328,0.2418,0.2380,0.2485,12.0860
ada,Ada Boost Classifier,0.9919,0.9526,0.1869,0.3263,0.2371,0.2333,0.2428,68.8560
gbc,Gradient Boosting Classifier,0.9882,0.9876,0.2272,0.2648,0.2327,0.2275,0.2336,387.5080
dt,Decision Tree Classifier,0.9561,0.6397,0.3189,0.2399,0.2473,0.2409,0.2461,13.6980
lda,Linear Discriminant Analysis,0.9704,0.8823,0.5865,0.1907,0.2770,0.2693,0.3148,19.8380
ridge,Ridge Classifier,0.9925,0.0000,0.0037,0.0500,0.0069,0.0056,0.0117,7.9640


CPU times: user 36min 46s, sys: 16.8 s, total: 37min 2s
Wall time: 1h 15min 54s


In [19]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, polynomial_features=True, polynomial_degree=2,polynomial_threshold=0.1)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9928,0.9912,0.1576,0.4314,0.2275,0.2246,0.2554,2.4480
et,Extra Trees Classifier,0.9829,0.9796,0.0403,0.3719,0.0563,0.0532,0.1019,0.1860
catboost,CatBoost Classifier,0.9924,0.9922,0.1393,0.3653,0.1978,0.1947,0.2198,14.3640
lightgbm,Light Gradient Boosting Machine,0.9910,0.9903,0.1943,0.2765,0.2255,0.2211,0.2259,0.3360
gbc,Gradient Boosting Classifier,0.9921,0.9912,0.0771,0.2672,0.1122,0.1096,0.1349,6.9380
dt,Decision Tree Classifier,0.9895,0.6219,0.2493,0.2384,0.2432,0.2379,0.2382,0.3940
rf,Random Forest Classifier,0.9927,0.9805,0.0221,0.2095,0.0376,0.0363,0.0617,0.6040
lda,Linear Discriminant Analysis,0.9659,0.9879,1.0000,0.2051,0.3348,0.3271,0.4378,0.1400
ada,Ada Boost Classifier,0.9912,0.9414,0.0696,0.1680,0.0954,0.0918,0.1022,1.4440
svm,SVM - Linear Kernel,0.9878,0.0000,0.0109,0.0137,0.0119,0.0059,0.0060,1.2000


CPU times: user 56.1 s, sys: 3.01 s, total: 59.1 s
Wall time: 2min 39s


In [21]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, polynomial_features=True, polynomial_degree=3,polynomial_threshold=0.1)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9927,0.9919,0.1688,0.4121,0.2359,0.2329,0.2581,2.3700
catboost,CatBoost Classifier,0.9925,0.9923,0.1687,0.3965,0.2318,0.2287,0.2523,13.5940
lightgbm,Light Gradient Boosting Machine,0.9917,0.9876,0.2311,0.3480,0.2710,0.2671,0.2759,0.6240
gbc,Gradient Boosting Classifier,0.9916,0.9881,0.1578,0.2962,0.2036,0.1998,0.2110,6.5700
dt,Decision Tree Classifier,0.9899,0.6257,0.2566,0.2585,0.2567,0.2517,0.2520,0.0940
ada,Ada Boost Classifier,0.9923,0.9363,0.0331,0.1434,0.0515,0.0494,0.0637,1.4460
lda,Linear Discriminant Analysis,0.9809,0.5348,0.0109,0.0857,0.0174,0.0141,0.0222,0.1360
et,Extra Trees Classifier,0.9812,0.9739,0.0037,0.0500,0.0069,0.0041,0.0087,0.2300
nb,Naive Bayes,0.0136,0.6782,0.9964,0.0068,0.0135,0.0000,0.0035,0.4760
qda,Quadratic Discriminant Analysis,0.0304,0.4968,0.9782,0.0068,0.0135,0.0000,0.0008,0.1060


CPU times: user 56.8 s, sys: 2.89 s, total: 59.6 s
Wall time: 2min 27s


In [23]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, polynomial_features=True, polynomial_degree=2,polynomial_threshold=0.01)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9925,0.9908,0.1393,0.3806,0.1984,0.1955,0.2234,1.9380
catboost,CatBoost Classifier,0.9923,0.9921,0.1394,0.3612,0.1956,0.1925,0.2174,13.8320
et,Extra Trees Classifier,0.9924,0.9824,0.0475,0.3335,0.0805,0.0784,0.1204,0.1540
lightgbm,Light Gradient Boosting Machine,0.9914,0.9907,0.2417,0.3332,0.2776,0.2734,0.2783,0.8240
gbc,Gradient Boosting Classifier,0.9922,0.9913,0.0804,0.3039,0.1231,0.1204,0.1500,5.2780
knn,K Neighbors Classifier,0.9926,0.7608,0.0512,0.2885,0.0858,0.0839,0.1176,0.1640
dt,Decision Tree Classifier,0.9890,0.6617,0.3298,0.2771,0.2936,0.2883,0.2931,0.0640
rf,Random Forest Classifier,0.9926,0.9834,0.0294,0.2363,0.0470,0.0456,0.0737,0.5340
lda,Linear Discriminant Analysis,0.9773,0.9879,1.0000,0.2312,0.3752,0.3682,0.4749,0.0960
qda,Quadratic Discriminant Analysis,0.9774,0.9848,0.9818,0.2298,0.3720,0.3651,0.4690,0.0860


CPU times: user 47.6 s, sys: 2.88 s, total: 50.5 s
Wall time: 2min 27s


In [24]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, trigonometry_features=True)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9925,0.9916,0.1688,0.3780,0.2304,0.2272,0.2474,1.9900
catboost,CatBoost Classifier,0.9923,0.9923,0.1574,0.3681,0.2157,0.2124,0.2343,13.5060
lightgbm,Light Gradient Boosting Machine,0.9915,0.9822,0.2238,0.3407,0.2625,0.2586,0.2680,0.7960
gbc,Gradient Boosting Classifier,0.9917,0.9881,0.1539,0.3058,0.2029,0.1991,0.2120,5.3000
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1620
dt,Decision Tree Classifier,0.9897,0.6310,0.2675,0.2558,0.2606,0.2554,0.2559,0.0860
ada,Ada Boost Classifier,0.9921,0.9342,0.0551,0.2061,0.0860,0.0834,0.1026,1.1840
rf,Random Forest Classifier,0.9883,0.9740,0.0073,0.2000,0.0140,0.0116,0.0350,0.8580
et,Extra Trees Classifier,0.9813,0.9746,0.0036,0.1000,0.0070,0.0043,0.0144,0.2000
nb,Naive Bayes,0.0136,0.5788,0.9964,0.0068,0.0135,0.0000,0.0035,0.0440


CPU times: user 40.8 s, sys: 1.74 s, total: 42.5 s
Wall time: 2min 13s


In [15]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, feature_selection=True, feature_selection_threshold=0.8)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9926,0.9918,0.1869,0.4115,0.2551,0.2519,0.2729,1.7340
catboost,CatBoost Classifier,0.9924,0.9924,0.1721,0.3886,0.2332,0.2299,0.2519,13.4640
lightgbm,Light Gradient Boosting Machine,0.9917,0.9850,0.1943,0.3175,0.2377,0.2338,0.2425,0.2900
gbc,Gradient Boosting Classifier,0.9917,0.9881,0.1576,0.3104,0.2072,0.2034,0.2163,5.1260
dt,Decision Tree Classifier,0.9900,0.6457,0.2966,0.2835,0.2884,0.2834,0.2842,0.3240
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.4240
ada,Ada Boost Classifier,0.9921,0.9342,0.0551,0.2061,0.0860,0.0834,0.1026,1.0380
et,Extra Trees Classifier,0.9828,0.9751,0.0073,0.2005,0.0080,0.0053,0.0237,0.2100
nb,Naive Bayes,0.0136,0.5788,0.9964,0.0068,0.0135,0.0000,0.0035,0.3320
qda,Quadratic Discriminant Analysis,0.0401,0.5237,0.9671,0.0068,0.0135,0.0000,-0.0016,0.3380


CPU times: user 48.4 s, sys: 2.68 s, total: 51.1 s
Wall time: 2min 18s


In [17]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, feature_selection=True, feature_selection_threshold=0.1)
best_model = compare_models(sort='Prec.',fold=3)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9929,0.9486,0.0293,0.4511,0.0523,0.0511,0.1041,0.5933
xgboost,Extreme Gradient Boosting,0.9926,0.9909,0.1465,0.3746,0.2105,0.2074,0.2310,0.8033
catboost,CatBoost Classifier,0.9925,0.9913,0.1245,0.3443,0.1827,0.1798,0.2038,11.8000
knn,K Neighbors Classifier,0.9930,0.6866,0.0330,0.3361,0.0594,0.0582,0.1015,0.2300
gbc,Gradient Boosting Classifier,0.9922,0.9906,0.1282,0.3145,0.1821,0.1789,0.1974,2.3633
lightgbm,Light Gradient Boosting Machine,0.9914,0.9872,0.1832,0.2959,0.2241,0.2200,0.2275,0.2333
dt,Decision Tree Classifier,0.9898,0.6347,0.2747,0.2595,0.2665,0.2614,0.2617,0.0800
rf,Random Forest Classifier,0.9931,0.9907,0.0073,0.2500,0.0142,0.0138,0.0411,0.3933
et,Extra Trees Classifier,0.9930,0.9911,0.0110,0.2421,0.0209,0.0203,0.0495,0.1900
lda,Linear Discriminant Analysis,0.9932,0.5666,0.0073,0.1333,0.0139,0.0137,0.0308,0.0800


CPU times: user 12.3 s, sys: 779 ms, total: 13.1 s
Wall time: 1min 1s


In [18]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, remove_multicollinearity=True, multicollinearity_threshold=0.9)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9929,0.9925,0.1762,0.4504,0.2513,0.2484,0.2774,12.8960
xgboost,Extreme Gradient Boosting,0.9924,0.9918,0.1725,0.3801,0.2312,0.2280,0.2488,1.2080
lightgbm,Light Gradient Boosting Machine,0.9921,0.9892,0.2236,0.3566,0.2688,0.2652,0.2751,0.8560
gbc,Gradient Boosting Classifier,0.9919,0.9893,0.1616,0.3149,0.2084,0.2048,0.2186,3.0620
dt,Decision Tree Classifier,0.9898,0.6368,0.2789,0.2620,0.2687,0.2636,0.2645,0.0600
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1320
et,Extra Trees Classifier,0.9929,0.9877,0.0036,0.2000,0.0071,0.0065,0.0258,0.1760
ada,Ada Boost Classifier,0.9922,0.9167,0.0183,0.1952,0.0333,0.0315,0.0569,0.7320
rf,Random Forest Classifier,0.9932,0.9896,0.0073,0.1333,0.0138,0.0135,0.0305,0.5920
lda,Linear Discriminant Analysis,0.9932,0.5523,0.0036,0.1000,0.0070,0.0069,0.0189,0.0660


CPU times: user 3min, sys: 8.28 s, total: 3min 8s
Wall time: 2min


In [19]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, remove_multicollinearity=True, multicollinearity_threshold=0.1)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9930,0.6459,0.0037,0.1000,0.0071,0.0066,0.0181,0.2060
gbc,Gradient Boosting Classifier,0.9901,0.6393,0.0074,0.0187,0.0106,0.0063,0.0071,0.6460
dt,Decision Tree Classifier,0.9858,0.5018,0.0111,0.0094,0.0102,0.0031,0.0031,0.0420
nb,Naive Bayes,0.0136,0.5801,0.9964,0.0068,0.0135,0.0000,0.0035,0.0160
qda,Quadratic Discriminant Analysis,0.0163,0.6736,0.9964,0.0068,0.0135,0.0001,0.0055,0.0140
svm,SVM - Linear Kernel,0.5014,0.0000,0.4978,0.0067,0.0132,-0.0002,-0.0001,0.0180
lr,Logistic Regression,0.5166,0.4559,0.3875,0.0054,0.0107,-0.0027,-0.0155,0.0160
knn,K Neighbors Classifier,0.9932,0.5126,0.0000,0.0000,0.0000,0.0000,0.0000,0.0720
ridge,Ridge Classifier,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0140
rf,Random Forest Classifier,0.9932,0.5603,0.0000,0.0000,0.0000,0.0000,0.0000,0.3140


CPU times: user 5.02 s, sys: 225 ms, total: 5.25 s
Wall time: 1min 24s


In [20]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, remove_multicollinearity=True, multicollinearity_threshold=0.99)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9923,0.9923,0.1758,0.3812,0.2352,0.2319,0.2522,13.5800
xgboost,Extreme Gradient Boosting,0.9924,0.9915,0.1505,0.3720,0.2079,0.2049,0.2294,1.8920
gbc,Gradient Boosting Classifier,0.9920,0.9880,0.1686,0.3499,0.2236,0.2201,0.2370,5.1620
lightgbm,Light Gradient Boosting Machine,0.9921,0.9888,0.2051,0.3488,0.2528,0.2492,0.2605,0.5760
dt,Decision Tree Classifier,0.9899,0.6293,0.2637,0.2670,0.2636,0.2585,0.2594,0.0820
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1480
ada,Ada Boost Classifier,0.9921,0.9342,0.0551,0.2061,0.0860,0.0834,0.1026,1.1420
et,Extra Trees Classifier,0.9860,0.9773,0.0036,0.2000,0.0071,0.0045,0.0230,0.2040
nb,Naive Bayes,0.0136,0.5788,0.9964,0.0068,0.0135,0.0000,0.0035,0.0440
qda,Quadratic Discriminant Analysis,0.0302,0.5195,0.9745,0.0068,0.0134,-0.0000,-0.0062,0.0760


CPU times: user 3min 8s, sys: 11.3 s, total: 3min 19s
Wall time: 2min 21s


In [21]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, remove_multicollinearity=True, multicollinearity_threshold=0.8)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9926,0.9927,0.1834,0.4243,0.2487,0.2457,0.2707,12.7920
xgboost,Extreme Gradient Boosting,0.9926,0.9921,0.1800,0.3945,0.2408,0.2378,0.2590,1.0900
rf,Random Forest Classifier,0.9933,0.9872,0.0147,0.3667,0.0283,0.0280,0.0727,0.5880
gbc,Gradient Boosting Classifier,0.9921,0.9885,0.2018,0.3608,0.2554,0.2518,0.2642,2.7400
lightgbm,Light Gradient Boosting Machine,0.9915,0.9872,0.1836,0.2911,0.2195,0.2157,0.2240,0.7320
dt,Decision Tree Classifier,0.9902,0.6497,0.3045,0.2879,0.2934,0.2884,0.2899,0.0380
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1240
ada,Ada Boost Classifier,0.9921,0.9550,0.0403,0.2512,0.0629,0.0607,0.0906,0.6360
lda,Linear Discriminant Analysis,0.9931,0.5569,0.0036,0.0333,0.0066,0.0063,0.0106,0.0580
qda,Quadratic Discriminant Analysis,0.1105,0.9616,0.9782,0.0074,0.0147,0.0013,0.0206,0.0460


CPU times: user 3min 7s, sys: 12 s, total: 3min 19s
Wall time: 1min 58s


In [22]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='linear',pca_components=0.99)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.0136,0.5073,0.9964,0.0068,0.0135,0.0000,0.0035,0.0100
qda,Quadratic Discriminant Analysis,0.0137,0.5073,0.9964,0.0068,0.0135,0.0000,0.0036,0.0120
lr,Logistic Regression,0.5975,0.5065,0.4000,0.0027,0.0054,-0.0028,-0.0013,0.0140
svm,SVM - Linear Kernel,0.7937,0.0000,0.2000,0.0014,0.0027,-0.0036,-0.0030,0.0140
knn,K Neighbors Classifier,0.9932,0.5052,0.0000,0.0000,0.0000,0.0000,0.0000,0.0580
dt,Decision Tree Classifier,0.9930,0.5138,0.0000,0.0000,0.0000,-0.0005,-0.0012,0.0220
ridge,Ridge Classifier,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0100
rf,Random Forest Classifier,0.9929,0.5117,0.0000,0.0000,0.0000,-0.0006,-0.0013,0.3940
ada,Ada Boost Classifier,0.9932,0.5418,0.0000,0.0000,0.0000,-0.0000,-0.0002,0.1100
gbc,Gradient Boosting Classifier,0.9923,0.5601,0.0000,0.0000,0.0000,-0.0016,-0.0024,0.2820


CPU times: user 6.08 s, sys: 658 ms, total: 6.74 s
Wall time: 1min 20s


In [15]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='linear',pca_components=0.5)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.0136,0.5073,0.9964,0.0068,0.0135,0.0000,0.0035,0.2960
qda,Quadratic Discriminant Analysis,0.0137,0.5073,0.9964,0.0068,0.0135,0.0000,0.0036,0.2700
lr,Logistic Regression,0.5975,0.5065,0.4000,0.0027,0.0054,-0.0028,-0.0013,0.5960
svm,SVM - Linear Kernel,0.7937,0.0000,0.2000,0.0014,0.0027,-0.0036,-0.0030,0.2600
knn,K Neighbors Classifier,0.9932,0.5052,0.0000,0.0000,0.0000,0.0000,0.0000,0.3480
dt,Decision Tree Classifier,0.9930,0.5138,0.0000,0.0000,0.0000,-0.0005,-0.0012,0.2700
ridge,Ridge Classifier,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0100
rf,Random Forest Classifier,0.9929,0.5117,0.0000,0.0000,0.0000,-0.0006,-0.0013,0.4340
ada,Ada Boost Classifier,0.9932,0.5418,0.0000,0.0000,0.0000,-0.0000,-0.0002,0.3340
gbc,Gradient Boosting Classifier,0.9923,0.5601,0.0000,0.0000,0.0000,-0.0016,-0.0024,0.2960


CPU times: user 6.77 s, sys: 1.27 s, total: 8.04 s
Wall time: 1min 45s


In [16]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='linear',pca_components=0.1)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.0136,0.5073,0.9964,0.0068,0.0135,0.0000,0.0035,0.0120
qda,Quadratic Discriminant Analysis,0.0137,0.5073,0.9964,0.0068,0.0135,0.0000,0.0036,0.0120
lr,Logistic Regression,0.5975,0.5065,0.4000,0.0027,0.0054,-0.0028,-0.0013,0.0140
svm,SVM - Linear Kernel,0.7937,0.0000,0.2000,0.0014,0.0027,-0.0036,-0.0030,0.0140
knn,K Neighbors Classifier,0.9932,0.5052,0.0000,0.0000,0.0000,0.0000,0.0000,0.0640
dt,Decision Tree Classifier,0.9930,0.5138,0.0000,0.0000,0.0000,-0.0005,-0.0012,0.0240
ridge,Ridge Classifier,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0120
rf,Random Forest Classifier,0.9929,0.5117,0.0000,0.0000,0.0000,-0.0006,-0.0013,0.4120
ada,Ada Boost Classifier,0.9932,0.5418,0.0000,0.0000,0.0000,-0.0000,-0.0002,0.1320
gbc,Gradient Boosting Classifier,0.9923,0.5601,0.0000,0.0000,0.0000,-0.0016,-0.0024,0.3020


CPU times: user 6.5 s, sys: 864 ms, total: 7.36 s
Wall time: 1min 16s


In [15]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='kernel',pca_components=5)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9914,0.5721,0.0221,0.0691,0.0334,0.0304,0.0354,0.7060
dt,Decision Tree Classifier,0.9866,0.5058,0.0185,0.0170,0.0177,0.0109,0.0110,0.3060
gbc,Gradient Boosting Classifier,0.9894,0.6321,0.0036,0.0061,0.0045,-0.0004,-0.0005,4.4840
lr,Logistic Regression,0.9932,0.6625,0.0000,0.0000,0.0000,0.0000,0.0000,0.3980
knn,K Neighbors Classifier,0.9932,0.5425,0.0000,0.0000,0.0000,-0.0000,-0.0002,585.8100
nb,Naive Bayes,0.9932,0.6544,0.0000,0.0000,0.0000,0.0000,0.0000,0.2740
svm,SVM - Linear Kernel,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2620
ridge,Ridge Classifier,0.9932,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2460
rf,Random Forest Classifier,0.9930,0.5733,0.0000,0.0000,0.0000,-0.0004,-0.0012,1.2460
qda,Quadratic Discriminant Analysis,0.9932,0.6576,0.0000,0.0000,0.0000,0.0000,0.0000,0.8520


CPU times: user 14min 35s, sys: 1min 38s, total: 16min 13s
Wall time: 56min 57s


In [16]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='incremental',pca_components=5)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9930,0.9568,0.0293,0.3083,0.0526,0.0516,0.0909,0.2620
lightgbm,Light Gradient Boosting Machine,0.9917,0.9875,0.1390,0.2806,0.1828,0.1791,0.1917,0.3200
et,Extra Trees Classifier,0.9923,0.9871,0.0842,0.2788,0.1288,0.1261,0.1498,0.1360
gbc,Gradient Boosting Classifier,0.9926,0.9800,0.0657,0.2786,0.1054,0.1031,0.1315,0.8300
dt,Decision Tree Classifier,0.9900,0.6273,0.2597,0.2625,0.2609,0.2559,0.2560,0.2820
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.4400
catboost,CatBoost Classifier,0.9916,0.9885,0.0951,0.2388,0.1328,0.1293,0.1448,12.0000
xgboost,Extreme Gradient Boosting,0.9917,0.9875,0.0948,0.2315,0.1306,0.1272,0.1418,0.6440
rf,Random Forest Classifier,0.9922,0.9855,0.0585,0.2143,0.0914,0.0888,0.1084,0.3160
nb,Naive Bayes,0.0136,0.5803,0.9964,0.0068,0.0135,0.0000,0.0035,0.3000


CPU times: user 7.38 s, sys: 908 ms, total: 8.29 s
Wall time: 1min 28s


In [52]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='incremental',pca_components=10)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9923,0.9744,0.0293,0.3775,0.0492,0.0474,0.0891,0.5260
ada,Ada Boost Classifier,0.9922,0.9429,0.0660,0.3350,0.1008,0.0985,0.1373,0.4380
gbc,Gradient Boosting Classifier,0.9918,0.9817,0.1246,0.2749,0.1699,0.1663,0.1804,1.6040
lightgbm,Light Gradient Boosting Machine,0.9910,0.9860,0.1832,0.2629,0.2157,0.2113,0.2149,0.3720
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.4360
xgboost,Extreme Gradient Boosting,0.9916,0.9885,0.1098,0.2560,0.1524,0.1488,0.1631,1.0520
dt,Decision Tree Classifier,0.9893,0.6270,0.2597,0.2370,0.2472,0.2419,0.2424,0.3480
catboost,CatBoost Classifier,0.9918,0.9895,0.0770,0.2076,0.1111,0.1079,0.1220,12.0840
et,Extra Trees Classifier,0.9923,0.9816,0.0182,0.1325,0.0312,0.0294,0.0456,0.1720
lda,Linear Discriminant Analysis,0.9929,0.5440,0.0036,0.1000,0.0070,0.0063,0.0182,0.0580


CPU times: user 15.3 s, sys: 877 ms, total: 16.2 s
Wall time: 1min 39s


In [53]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='incremental',pca_components=15)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9922,0.9885,0.1138,0.3335,0.1605,0.1575,0.1845,1.2640
gbc,Gradient Boosting Classifier,0.9918,0.9843,0.1064,0.2974,0.1469,0.1437,0.1674,2.6220
lightgbm,Light Gradient Boosting Machine,0.9911,0.9861,0.1688,0.2633,0.2032,0.1990,0.2051,0.3980
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1280
dt,Decision Tree Classifier,0.9893,0.6199,0.2454,0.2280,0.2356,0.2302,0.2308,0.0820
ada,Ada Boost Classifier,0.9917,0.9401,0.0514,0.2255,0.0727,0.0700,0.0941,0.6320
catboost,CatBoost Classifier,0.9919,0.9894,0.0844,0.2221,0.1180,0.1150,0.1304,12.3720
rf,Random Forest Classifier,0.9929,0.9746,0.0074,0.0333,0.0121,0.0115,0.0145,0.6000
et,Extra Trees Classifier,0.9928,0.9792,0.0037,0.0154,0.0060,0.0051,0.0060,0.1640
qda,Quadratic Discriminant Analysis,0.0457,0.9651,0.9891,0.0070,0.0138,0.0004,0.0115,0.0440


CPU times: user 32.3 s, sys: 1.86 s, total: 34.1 s
Wall time: 1min 44s


In [54]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, pca=True, pca_method='incremental',pca_components=20)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9923,0.9899,0.1029,0.3085,0.1516,0.1488,0.1730,13.3520
ada,Ada Boost Classifier,0.9921,0.9434,0.0735,0.2662,0.1014,0.0989,0.1256,0.8180
dt,Decision Tree Classifier,0.9894,0.6438,0.2935,0.2604,0.2725,0.2672,0.2694,0.0980
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.1340
gbc,Gradient Boosting Classifier,0.9915,0.9848,0.1098,0.2400,0.1488,0.1451,0.1573,3.8180
lightgbm,Light Gradient Boosting Machine,0.9903,0.9891,0.1795,0.2309,0.2003,0.1955,0.1979,0.5800
xgboost,Extreme Gradient Boosting,0.9917,0.9888,0.0770,0.2006,0.1064,0.1033,0.1173,1.7060
rf,Random Forest Classifier,0.9931,0.9699,0.0074,0.1250,0.0136,0.0131,0.0280,0.8420
lda,Linear Discriminant Analysis,0.9929,0.5538,0.0036,0.0250,0.0063,0.0057,0.0085,0.0580
qda,Quadratic Discriminant Analysis,0.0937,0.9640,0.9855,0.0073,0.0145,0.0011,0.0196,0.0540


CPU times: user 3min 2s, sys: 9.69 s, total: 3min 11s
Wall time: 2min 11s


In [55]:
%%time
# Let's play with interactions ==> Très long
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, create_clusters=True, cluster_iter=20)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9925,0.9923,0.1832,0.4177,0.2492,0.2460,0.2700,13.4400
xgboost,Extreme Gradient Boosting,0.9927,0.9916,0.1762,0.4060,0.2427,0.2396,0.2624,2.0200
lightgbm,Light Gradient Boosting Machine,0.9915,0.9822,0.2238,0.3407,0.2625,0.2586,0.2680,0.2160
gbc,Gradient Boosting Classifier,0.9918,0.9881,0.1576,0.3149,0.2084,0.2047,0.2180,5.0000
dt,Decision Tree Classifier,0.9899,0.6366,0.2785,0.2699,0.2726,0.2676,0.2684,0.3440
knn,K Neighbors Classifier,0.9928,0.7084,0.0329,0.2567,0.0579,0.0564,0.0889,0.4340
ada,Ada Boost Classifier,0.9921,0.9342,0.0551,0.2061,0.0860,0.0834,0.1026,1.3880
rf,Random Forest Classifier,0.9843,0.9741,0.0036,0.2000,0.0071,0.0046,0.0230,1.0180
nb,Naive Bayes,0.0136,0.5786,0.9964,0.0068,0.0135,0.0000,0.0035,0.0460
svm,SVM - Linear Kernel,0.5014,0.0000,0.4978,0.0067,0.0132,-0.0002,-0.0001,0.3480


CPU times: user 31min 57s, sys: 5min 49s, total: 37min 47s
Wall time: 8min 38s


In [ ]:
######################################@

In [16]:
%%time
# Let's test a combination
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, create_clusters=True, cluster_iter=20, normalize=True, normalize_method='maxabs', transformation=True, transformation_method='quantile', feature_interaction=True, feature_ratio=True,interaction_threshold=0.01, polynomial_features=True, polynomial_degree=2,polynomial_threshold=0.1, feature_selection=True, feature_selection_threshold=0.1, remove_multicollinearity=True, multicollinearity_threshold=0.9,remove_outliers=True,outliers_threshold=0.05)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9953,0.9617,0.1191,0.3113,0.1680,0.1661,0.1878,0.5920
svm,SVM - Linear Kernel,0.9943,0.0000,0.1198,0.2926,0.1029,0.1010,0.1304,0.0780
dt,Decision Tree Classifier,0.9936,0.6440,0.2916,0.2635,0.2710,0.2679,0.2711,0.2980
lda,Linear Discriminant Analysis,0.9875,0.9939,0.9600,0.2392,0.3819,0.3780,0.4749,0.0580
knn,K Neighbors Classifier,0.9950,0.9125,0.1062,0.2356,0.1398,0.1377,0.1518,1.5940
nb,Naive Bayes,0.9868,0.9934,1.0000,0.2325,0.3765,0.3725,0.4782,0.3000
gbc,Gradient Boosting Classifier,0.9950,0.9678,0.1256,0.2310,0.1620,0.1597,0.1675,1.5860
lightgbm,Light Gradient Boosting Machine,0.9942,0.9872,0.1856,0.2290,0.2024,0.1995,0.2020,0.4060
xgboost,Extreme Gradient Boosting,0.9949,0.9940,0.0994,0.2229,0.1319,0.1297,0.1433,0.7300
rf,Random Forest Classifier,0.9956,0.9943,0.0331,0.2071,0.0563,0.0552,0.0801,0.4660


CPU times: user 30min 43s, sys: 6min 9s, total: 36min 53s
Wall time: 7min 30s


In [195]:
%%time
# Let's test a combination
naive = setup(data = features_train.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), test_data = features_test.drop(['TRACKER','Date','Symbol'],axis=1).astype(np.float32), target = 'Valid', session_id=26, create_clusters=True, cluster_iter=20, normalize=True, normalize_method='maxabs', transformation=True, transformation_method='quantile', feature_interaction=True, feature_ratio=True,interaction_threshold=0.01, polynomial_features=True, polynomial_degree=2,polynomial_threshold=0.1, feature_selection=True, feature_selection_threshold=0.1, remove_multicollinearity=True, multicollinearity_threshold=0.9)
best_model = compare_models(sort='Prec.',fold=5)
engine.say("Classification with features interaction done")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9931,0.9918,0.0623,0.4761,0.1086,0.1071,0.1676,0.1520
lr,Logistic Regression,0.9928,0.9923,0.0914,0.4021,0.1459,0.1437,0.1857,1.1140
ridge,Ridge Classifier,0.9932,0.0000,0.0366,0.3800,0.0667,0.0657,0.1161,0.2840
xgboost,Extreme Gradient Boosting,0.9923,0.9912,0.1870,0.3738,0.2482,0.2448,0.2603,1.0020
rf,Random Forest Classifier,0.9930,0.9883,0.0624,0.3640,0.1046,0.1030,0.1458,0.2280
catboost,CatBoost Classifier,0.9925,0.9920,0.1321,0.3535,0.1910,0.1880,0.2120,12.7640
lightgbm,Light Gradient Boosting Machine,0.9915,0.9905,0.2418,0.3416,0.2801,0.2760,0.2816,0.4460
knn,K Neighbors Classifier,0.9923,0.8959,0.1503,0.3400,0.2078,0.2045,0.2222,2.4420
gbc,Gradient Boosting Classifier,0.9921,0.9916,0.1610,0.3357,0.2168,0.2133,0.2284,1.8180
lda,Linear Discriminant Analysis,0.9859,0.9890,0.8681,0.3090,0.4552,0.4497,0.5127,0.0720


CPU times: user 34min 6s, sys: 6min 37s, total: 40min 43s
Wall time: 8min 19s


In [201]:
features_oos = features[(features.index>=_stop)].drop(['TRACKER','Date','Symbol','Valid'],axis=1).astype(np.float32)

In [202]:
best_model.predict(features_oos)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [9]:
%%time
_ticker = x.replace('/','')
_start = '2017-01-01'
_mid = '2018-08-31'
_stop = '2019-12-31'
features = features.dropna()
features = features[features.Symbol==_ticker]
features = features[features.Signal!=0]
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)
features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[features.Date>_stop]

CPU times: user 6.4 s, sys: 4.33 s, total: 10.7 s
Wall time: 10.7 s


In [27]:
%%time
#######################################
##### S C A L I N G ###################
#######################################
_ticker = x.replace('/','')
_start = '2017-01-01'
_mid = '2018-08-31'
_stop = '2019-12-31'
features_train,features_test,features_oos = scaling(features,_ticker,TRACKER,_start,_mid,_stop,scaler)

CPU times: user 82.7 ms, sys: 3.9 ms, total: 86.6 ms
Wall time: 85.4 ms


In [10]:
#####################################
########### s v m . S V C ###########
#####################################

In [11]:
%%time
##### Construction du modèle d'apprentissage => BUY et SELL sont les OUTPUT
features_train['Valid'] = np.where((features_train.Signal!=0)&(features_train.TRACKER==1),1,0)
features_test['Valid'] = np.where((features_test.Signal!=0)&(features_test.TRACKER==1),1,0)

CPU times: user 3.93 ms, sys: 1.35 ms, total: 5.28 ms
Wall time: 4.14 ms


In [12]:
%%time
data = features_train[features_train.Symbol==_ticker].drop(['Date','Symbol','TRACKER'],axis=1).astype(np.float32)
data_test = features_test[features_test.Symbol==_ticker].drop(['Date','Symbol','TRACKER'],axis=1).astype(np.float32)

CPU times: user 4.95 ms, sys: 1.92 ms, total: 6.87 ms
Wall time: 5.84 ms


In [13]:
from pycaret.classification import *

exp_clf102 = setup(data = data, test_data = data_test, target = 'Valid', session_id=26) # ,fix_imbalance=True,fix_imbalance_method=True)

In [17]:
%%time
exp_clf101 = setup(data = data, test_data = data_test, target = 'Valid', session_id=26,feature_selection=True,multicollinearity_threshold=0.99,pca=False,pca_method='kernel',pca_components=10,ignore_low_variance=True,remove_outliers=True,outliers_threshold=0.05,polynomial_features=True,trigonometry_features=True)

best_model = compare_models(sort='Prec.',fold=5)
engine.say("בסדר גמור")
engine.runAndWait()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7460,0.5908,0.1073,0.3747,0.1631,0.0604,0.0814,0.3820
xgboost,Extreme Gradient Boosting,0.7354,0.6003,0.1265,0.3296,0.1790,0.0571,0.0686,0.3460
catboost,CatBoost Classifier,0.7566,0.6356,0.0152,0.2917,0.0285,-0.0075,-0.0009,2.8240
rf,Random Forest Classifier,0.7487,0.5585,0.0422,0.2800,0.0710,0.0053,0.0142,0.0860
gbc,Gradient Boosting Classifier,0.7336,0.5340,0.0843,0.2506,0.1190,0.0146,0.0162,0.2440
qda,Quadratic Discriminant Analysis,0.5761,0.4885,0.3718,0.2433,0.2897,0.0131,0.0114,0.0100
et,Extra Trees Classifier,0.7434,0.5505,0.0538,0.2256,0.0808,0.0080,0.0157,0.0720
ada,Ada Boost Classifier,0.7027,0.4979,0.1073,0.2187,0.1410,-0.0131,-0.0131,0.0680
nb,Naive Bayes,0.6823,0.4884,0.1229,0.2166,0.1475,-0.0221,-0.0232,0.0100
dt,Decision Tree Classifier,0.5938,0.4786,0.2643,0.2097,0.2305,-0.0355,-0.0377,0.0140


CPU times: user 18.1 s, sys: 2.17 s, total: 20.2 s
Wall time: 29.5 s


In [138]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=26, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [22]:

from scipy.signal import argrelextrema

n = 5  # number of points to be checked before and after

# Find local peaks

daily_all['min'] = daily_all.iloc[argrelextrema(daily_all.Close.values, np.less_equal,
                    order=n)[0]]['Close']
daily_all['max'] = daily_all.iloc[argrelextrema(daily_all.Close.values, np.greater_equal,
                    order=n)[0]]['Close']

# Plot results

plt.scatter(daily_all.index, daily_all['min'], c='r')
plt.scatter(daily_all.index, daily_all['max'], c='g')
plt.plot(daily_all.index, daily_all['Close'])
plt.show()

ValueError: cannot reindex from a duplicate axis